# Matrix Factorization

In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
from datetime import datetime
from numpy.linalg import svd
from sklearn.decomposition import TruncatedSVD

import csv
import os
import sys

## Utils

In [29]:
DATA_PATH = '/opt/ml/input/data/train_dataset/'
TRAIN_DATA = 'cv_train_FE.pkl'
VALID_DATA = 'cv_valid_FE.pkl'
TEST_DATA = 'test_FE.pkl'
K=10

In [4]:
def drop_unnamed(df):
    # Drop index column in df:
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
        print("drop Unnamed: 0 column")
    return df

In [3]:
def convert_time(s):
    # Convert datetime64 to int
    timestamp = time.mktime(datetime.strptime(str(s), '%Y-%m-%d %H:%M:%S').timetuple())
    return int(timestamp)

In [9]:
def csv_to_pkl():
    # Convert csv dataset to pkl dataset for performance.    
    if TRAIN_DATA[-3:]=='csv' : train_df.to_pickle(os.path.join(DATA_PATH,'cv_train_data_FE.pkl'))
    if VALID_DATA[-3:]=='csv' : valid_df.to_pickle(os.path.join(DATA_PATH,'cv_valid_data_FE.pkl'))
    if TEST_DATA[-3:]=='csv' : test_df.to_pickle(os.path.join(DATA_PATH,'cv_test_data_FE.pkl'))
csv_to_pkl()

In [8]:
def cache_time(df, name):
    # Convert datetime64 to int, then save it.
    if df['Timestamp'].dtype == object:
        print(df['Timestamp'].dtype, df['Timestamp'].head(1))
        print("Processing Timestamp...")
        df['Timestamp'] = df['Timestamp'].apply(convert_time)
        print("Processing Timestamp done")
        df.to_pickle(os.path.join(DATA_PATH,f'{name}.pkl'))
    return df
train_df = cache_time(train_df, 'cv_train_FE')
valid_df = cache_time(valid_df, 'cv_valid_FE')
test_df = cache_time(test_df, 'test_FE')

## Get Data and Concat Datasets

In [30]:
# Load dataset from disk
get_train_data = pd.read_csv if TRAIN_DATA[-3:]=='csv' else pd.read_pickle
get_valid_data = pd.read_csv if VALID_DATA[-3:]=='csv' else pd.read_pickle
get_test_data = pd.read_csv if TEST_DATA[-3:]=='csv' else pd.read_pickle

train_df = get_train_data(os.path.join(DATA_PATH, TRAIN_DATA))
valid_df = get_valid_data(os.path.join(DATA_PATH, VALID_DATA))
test_df = get_test_data(os.path.join(DATA_PATH, TEST_DATA))

In [31]:
original_train = pd.read_csv(os.path.join(DATA_PATH, 'train_data.csv'))
original_test = pd.read_csv(os.path.join(DATA_PATH, 'test_data.csv'))

In [32]:
# Check dataset length
len(original_train),len(original_test),len(original_train) == len(train_df) + len(valid_df),len(original_test)==len(test_df)

(2266586, 260114, True, True)

In [12]:
train_df.columns, train_df.columns == valid_df.columns, len(train_df.columns)

(Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
        'KnowledgeTag', 'hour', 'dow', 'elapsed', 'grade', 'mid',
        'problem_number', 'test_mean', 'test_sum', 'tag_mean', 'tag_sum',
        'ass_mean', 'ass_sum', 'prb_mean', 'prb_sum', 'hour_mean', 'hour_sum',
        'dow_mean', 'dow_sum', 'tag_elp', 'tag_elp_o', 'tag_elp_x', 'ass_elp',
        'ass_elp_o', 'ass_elp_x', 'prb_elp', 'prb_elp_o', 'prb_elp_x',
        'user_correct_answer', 'user_total_answer', 'user_acc', 'Grade_o',
        'GradeCount', 'GradeAcc', 'GradeElp', 'GradeMElp', 'problem_count',
        'tag_count', 'RepeatedTime', 'prior_KnowledgeTag_frequency',
        'problem_position', 'solve_order', 'retest', 'solved_disorder',
        'last_problem', 'answer_delta', 'tag_delta', 'test_delta',
        'assess_delta', 'left_asymptote', 'elo_prob', 'cum_correct',
        'prior_relative_assess_ac_sum', 'prior_relative_answer_ac_sum',
        'prior_relative_tag_ac_sum', 'prior_relative_tes

In [13]:
test_df.columns, len(test_df.columns)

(Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
        'KnowledgeTag', 'hour', 'dow', 'elapsed', 'grade', 'mid',
        'problem_number', 'test_mean', 'test_sum', 'tag_mean', 'tag_sum',
        'ass_mean', 'ass_sum', 'prb_mean', 'prb_sum', 'hour_mean', 'hour_sum',
        'dow_mean', 'dow_sum', 'tag_elp', 'tag_elp_o', 'tag_elp_x', 'ass_elp',
        'ass_elp_o', 'ass_elp_x', 'prb_elp', 'prb_elp_o', 'prb_elp_x',
        'user_correct_answer', 'user_total_answer', 'user_acc', 'Grade_o',
        'GradeCount', 'GradeAcc', 'GradeElp', 'GradeMElp', 'problem_count',
        'tag_count', 'RepeatedTime', 'prior_KnowledgeTag_frequency',
        'problem_position', 'solve_order', 'retest', 'solved_disorder',
        'last_problem', 'answer_delta', 'tag_delta', 'test_delta',
        'assess_delta', 'left_asymptote', 'elo_prob',
        'prior_relative_assess_ac_sum', 'prior_relative_answer_ac_sum',
        'prior_relative_tag_ac_sum', 'prior_relative_test_ac_sum'],
   

In [14]:
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,hour,dow,elapsed,grade,...,answer_delta,tag_delta,test_delta,assess_delta,left_asymptote,elo_prob,prior_relative_assess_ac_sum,prior_relative_answer_ac_sum,prior_relative_tag_ac_sum,prior_relative_test_ac_sum
0,3,A050023001,A050000023,1,1578567391,2626,10,3,0.0,5,...,0.000000,0.000000,0.000000,0.000000,0,0.752296,0.000000,0.000000,0.000000,0.000000
1,3,A050023002,A050000023,1,1578567417,2626,10,3,26.0,5,...,0.337382,0.412903,0.526786,0.250000,0,0.416693,0.250000,0.337382,0.412903,0.526786
2,3,A050023003,A050000023,0,1578567511,2625,10,3,94.0,5,...,0.337382,0.412903,0.526786,0.437500,0,0.281076,0.687500,0.674764,0.825806,1.053571
3,3,A050023004,A050000023,0,1578567516,2625,10,3,5.0,5,...,-0.662618,-0.588517,-0.473214,-0.343750,0,0.382929,0.343750,0.012146,0.237290,0.580357
4,3,A050023006,A050000023,0,1578567523,2623,10,3,7.0,5,...,-0.662618,-0.588517,-0.473214,-0.500000,0,0.166432,-0.156250,-0.650472,-0.351227,0.107143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,1602716843,8832,23,2,0.0,4,...,0.355276,0.136364,0.255208,0.125000,0,0.132075,-0.711993,0.908039,0.370093,-0.711993
260110,7439,A040130002,A040000130,1,1602716861,8832,23,2,18.0,4,...,-0.644724,0.136364,-0.604762,-0.380952,0,0.410814,-1.092946,0.263315,0.506456,-1.316755
260111,7439,A040130003,A040000130,1,1602716882,8244,23,2,21.0,4,...,0.355276,0.136364,0.395238,0.476190,0,0.621880,-0.616755,0.618591,0.642820,-0.921517
260112,7439,A040130004,A040000130,1,1602716971,8244,23,2,89.0,4,...,0.355276,-0.776087,0.395238,0.238095,0,0.760312,-0.378660,0.973867,-0.133267,-0.526279


In [16]:
# Drop unavailable columns
# raise RuntimeError("Preventing Error!")
unavailable = ['cum_correct']
train_df = train_df.drop(columns=unavailable, errors='ignore')
valid_df = valid_df.drop(columns=unavailable, errors='ignore')
test_df = test_df.drop(columns=unavailable, errors='ignore')

In [17]:
# Check all the dataset's columns are same.
train_df.columns == valid_df.columns, valid_df.columns == test_df.columns, test_df.columns == train_df.columns

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True]),
 array([ True,  True,  True,  True,  True,  

In [18]:
# Add temporal Index and Table column for spliting.
# 0: train, 1: valid, 2: test
train_df['table'] = 0
valid_df['table'] = 1
test_df['table'] = 2

train_df['temp_idx'] = train_df.index
valid_df['temp_idx'] = valid_df.index
test_df['temp_idx'] = test_df.index

In [19]:
# Get masked test rows
masked_test_df = test_df[test_df['answerCode'] == -1]
unmasked_test_df = test_df[test_df['answerCode'] != -1]
len(masked_test_df)

744

In [20]:
# Concat all datasets without masked rows.
whole_df = pd.concat([train_df, valid_df, unmasked_test_df], ignore_index=True)

In [21]:
# Is SVD avaliable? Unknown userID and Unknown assessmentItemID in test dataset will cause problem.
print("problem? : ")
masked_userID = masked_test_df['userID'].unique()
masked_assessmentItemID = masked_test_df['assessmentItemID'].unique()
if any(user not in whole_df['userID'].unique() for user in tqdm(masked_userID)) or any(user not in whole_df['assessmentItemID'].unique() for user in tqdm(masked_assessmentItemID)) : 
    raise RuntimeError(f'TSVD Feature Unavailable.')
print("nope, Happy hacking!")

problem? : 


  0%|          | 0/744 [00:00<?, ?it/s]

  0%|          | 0/444 [00:00<?, ?it/s]

nope, Happy hacking!


In [22]:
# Concat all df as one.
whole_df = pd.concat([whole_df, masked_test_df], ignore_index=True)

In [23]:
len(whole_df), len(whole_df) == len(train_df) + len(valid_df) + len(test_df)

(2526700, True)

In [24]:
len(whole_df['userID'].unique()), len(whole_df['assessmentItemID'].unique())

(7442, 9454)

## Truncated SVD
- 가장 중요한 K개의 잠재요소만 가져와서 Feature로 삼는 방법

In [25]:
def save_TSVD(k, df):
    # Add lf function in Dataframe.

    # Careate Pivot Table
    print("Create Pivot Table")
    ans_df = df[df['answerCode'] != -1].groupby(['userID', 'assessmentItemID']).answerCode.sum().reset_index()
    pivot_df = ans_df.pivot(index='userID', columns='assessmentItemID', values='answerCode').fillna(0)

    # fit SVD    
    print("fit SVD")
    svd2 = TruncatedSVD(n_components=k)
    svd2.fit(pivot_df)
    user_hid = svd2.transform(pivot_df)
    print("유저 잠재요인 : ", len(user_hid), len(user_hid[0]))
    svd2.fit(pivot_df.T)
    problems_hid = svd2.transform(pivot_df.T)
    print("문제 잠재요인 : ", len(problems_hid), len(problems_hid[0]))
    users = pivot_df.index.values
    problems = pivot_df.columns.values

    # 유저 잠재 요인 - U
    user_latent_factor = {}

    for i, user in enumerate(users):
        user_latent_factor[user] = user_hid[i]

    # 문제 잠재 요인 - V
    problems_latent_factor = {}

    for i, problem in enumerate(problems):
        problems_latent_factor[problem] = problems_hid[i]

    print("assessmentItemID mapping")
    nested_problems_lf = df['assessmentItemID'].map(problems_latent_factor).values
    problem_lf = np.concatenate(nested_problems_lf, 0).reshape(-1, len(nested_problems_lf[0]))
    # Add feature
    print("Add feature")
    df[[f'assessmentItemID_lf{i + 1}' for i in tqdm(range(k))]] = problem_lf
    return df



In [26]:
# truncatedSVD
new_df = save_TSVD(K, whole_df)
new_df

Create Pivot Table
fit SVD
유저 잠재요인 :  7442 10
문제 잠재요인 :  9454 10
assessmentItemID mapping
Add feature


  0%|          | 0/10 [00:00<?, ?it/s]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,hour,dow,elapsed,grade,...,assessmentItemID_lf1,assessmentItemID_lf2,assessmentItemID_lf3,assessmentItemID_lf4,assessmentItemID_lf5,assessmentItemID_lf6,assessmentItemID_lf7,assessmentItemID_lf8,assessmentItemID_lf9,assessmentItemID_lf10
0,0,A060001001,A060000001,1,1585009031,7224,0,1,0.0,6,...,4.603269,-1.805121,-5.570956,7.150968,-0.447569,-1.895929,-0.744443,-0.781416,-0.003935,0.070115
1,0,A060001002,A060000001,1,1585009034,7225,0,1,3.0,6,...,4.547978,-1.742128,-5.559953,7.053310,-0.449389,-1.861193,-0.735543,-0.779536,-0.001006,0.075968
2,0,A060001003,A060000001,1,1585009042,7225,0,1,8.0,6,...,4.338867,-1.651416,-5.316583,6.834635,-0.451975,-1.752899,-0.646381,-0.687427,-0.000791,0.115197
3,0,A060001004,A060000001,1,1585009049,7225,0,1,7.0,6,...,4.595342,-1.797499,-5.573008,7.150943,-0.453717,-1.892042,-0.745691,-0.778797,-0.002439,0.072494
4,0,A060001005,A060000001,1,1585009056,7225,0,1,7.0,6,...,4.482671,-1.714868,-5.472689,6.943464,-0.438975,-1.824496,-0.725131,-0.789132,0.012729,0.077317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7395,A040122005,A040000122,-1,1599530720,10615,2,1,2.0,4,...,1.302587,-0.354698,-0.371282,-0.571389,-0.188631,1.780302,-0.904812,-0.700505,0.092521,0.054238
2526696,7404,A030111005,A030000111,-1,1602582558,7636,9,1,107.0,3,...,4.310803,-2.086544,-3.398213,-5.089712,7.166411,-1.623552,-0.501869,-0.839058,-0.063924,0.314202
2526697,7416,A050193004,A050000193,-1,1601779481,10402,2,6,24.0,5,...,1.639639,-0.373915,-0.580215,-1.050618,-1.682823,-0.761719,-0.289506,0.458620,-0.383207,0.208749
2526698,7417,A050193004,A050000193,-1,1599397755,10402,13,6,21.0,5,...,1.639639,-0.373915,-0.580215,-1.050618,-1.682823,-0.761719,-0.289506,0.458620,-0.383207,0.208749


## Split dataframe and Save it.

In [27]:
train_df = whole_df[whole_df['table']==0].sort_values(by='temp_idx').drop(columns=['table', 'temp_idx']).to_pickle(os.path.join(DATA_PATH,f'cv_train_data_FE_MF{K}.pkl'))
valid_df = whole_df[whole_df['table']==1].sort_values(by='temp_idx').drop(columns=['table', 'temp_idx']).to_pickle(os.path.join(DATA_PATH,f'cv_valid_data_FE_MF{K}.pkl'))
test_df = whole_df[whole_df['table']==2].sort_values(by='temp_idx').drop(columns=['table', 'temp_idx']).to_pickle(os.path.join(DATA_PATH,f'cv_test_data_FE_MF{K}.pkl'))



## SVD
memory overflow! unavaliable!  

In [24]:
# untruncated svd
raise RuntimeError("Unavailable!")
U, sigma, VT = svd(np.array(test_pivot_df))

RuntimeError: Unavailable!

In [ ]:
smat= np.zeros((len(U),len(VT)))
smat[:len(U),:len(U)] = np.diag(sigma)


In [ ]:
users = test_pivot_df.index.values
problems = test_pivot_df.columns.values

# 유저 잠재 요인 - U
user_latent_factor = {}

# scaling
U = U @ np.diag(sigma)

for i, user in enumerate(users):
    user_latent_factor[user] = U[i]

# 문제 잠재 요인 - V
problems_latent_factor = {}

# scaling
# VT = np.diag(sigma) @ VT
VT = smat @ VT

for i, problem in enumerate(problems):
    problems_latent_factor[problem] = VT.T[i]

In [9]:
# 문제 잠재 요인들을 각 문제에 mapping
nested_problems_lf = test_svd_df['assessmentItemID'].map(problems_latent_factor).values


TypeError: 'TruncatedSVD' object is not callable

In [ ]:
# nested numpy array를 하나의 numpy array로 변환
problem_lf = np.concatenate(nested_problems_lf, 0).reshape(-1, nested_problems_lf[0])


In [ ]:

# # 문제 잠재 요인을 기존 pandas DataFrame에 추가
test_fe_df[[f'assessmentItemID_lf{i + 1}' for i in range(3)]] = problem_lf

In [ ]:
test_fe_df